In [1]:
import os
import wandb
import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.metrics import f1_score, classification_report

import torch
#from torch.utils.data import Dataset
from datasets import Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments, pipeline
from peft import LoraConfig
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM

/ext3/miniconda3/envs/owl-botu/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
wandb.init(project="LLM-bloomz-clm")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: sargunnagpal (a-is-all-we-need). Use `wandb login --relogin` to force relogin


## Load Data & Create Prompt

**Prompt Example**:

You are an expert in hate speech detection. Offensive tweets are defined as tweets containing profane words, sarcastic remarks, insults, slanders or slurs. These can have a potentially harmful effect on a given target. Classify the following input tweet as Offensive or Non-Offensive.

`###` Input: <tweet>

`###` Response: Offensive

In [3]:
system_prompt = "You are an expert in hate speech detection. Offensive tweets are defined as tweets containing profane words, sarcastic remarks, insults, slanders or slurs. These can have a potentially harmful effect on a given target. Classify the following input tweet as Offensive or Non-Offensive."
label_map = {1: "Offensive", 0: "Non-Offensive"}

In [4]:
def prepare_prompt(row, train=True):
    # Data Format -- https://huggingface.co/datasets/vicgalle/alpaca-gpt4?row=0
    prompt = system_prompt + "\n\n ### Input: " + row["tweet_text"] + "\n\n ### Response: "
    if train:
         prompt = prompt + label_map[row["offense"]] # Add label
    return prompt

In [5]:
train_df = pd.read_csv("data/splits/train.csv")
val_df = pd.read_csv("data/splits/val.csv")

# Prompt Column
train_df["text"] = train_df.apply(lambda row: prepare_prompt(row), axis=1)
val_df["text"] = val_df.apply(lambda row: prepare_prompt(row), axis=1)

# HF Datasets
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)

train_dataset

Dataset({
    features: ['Unnamed: 0', 'id', 'tweet_id', 'aggression', 'offense', 'codemixed', 'tweet_text', 'text'],
    num_rows: 6809
})

## Load Model

In [6]:
checkpoint = "bigscience/bloomz-7b1-mt" # Recommended for prompting in non-English (7.1B params)

In [7]:
# 4-bit Quantization (config passed to AutoModel.from_pretrained)
quant_config = BitsAndBytesConfig(
                            load_in_4bit = True,
                            bnb_4bit_quant_type = "nf4", # Normal Float 4-bits
                            bnb_4bit_compute_dtype = getattr(torch, "float16"), # torch.float16
                            bnb_4bit_use_double_quant = False,
                        )

In [8]:
model = AutoModelForCausalLM.from_pretrained(checkpoint, quantization_config=quant_config)

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
# tokenizer.pad_token = tokenizer.eos_token # Tokenizer does not have a padding token, but need it for batching
# tokenizer.padding_side = "left"

# model.config.use_cache = False
# model.config.pretraining_tp = 1

In [9]:
def print_trainable_parameters(model):
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"Trainable params = {trainable_params} | All params = {all_param} | Trainable % = {100 * trainable_params / all_param:.2f}"
    )

In [10]:
print_trainable_parameters(model)

Trainable params = 1028112384 | All params = 4049117184 | Trainable % = 25.39


## Configure Training Args & PEFT LoRA adapters

In [11]:
# Parameter Efficient Fine-Tuning with LoRA adapters. (config passed to SFT Trainer)
peft_config = LoraConfig(
                    lora_alpha=16,
                    lora_dropout=0.1,
                    r=64,
                    bias="none",
                    task_type="CAUSAL_LM",
                    )
# Note: If doing classification, need to wrap model in peft class

In [12]:
training_params = TrainingArguments(
                            output_dir="./results_bloomz2",
                            num_train_epochs=5,
                            per_device_train_batch_size=4,
                            gradient_accumulation_steps=1,
                            optim="paged_adamw_32bit",
                            save_steps=25,
                            logging_steps=25,
                            learning_rate=5e-5,
                            weight_decay=0.001,
                            fp16=False,
                            bf16=False,
                            max_grad_norm=0.3,
                            max_steps=-1,
                            warmup_ratio=0.03,
                            group_by_length=True,
                            evaluation_strategy="epoch",
                            save_strategy="epoch",
                            load_best_model_at_end=True,
                            lr_scheduler_type="constant"
                        )

## Train with SFT Trainer

#### Response Template for CompletionOnlyLM

DataCollatorForCompletionOnlyLM: Only train the model to generate the response (instead of the full sequence)
https://huggingface.co/docs/trl/sft_trainer#advanced-usage

Response Template not recognized fix -- https://github.com/huggingface/trl/issues/980
https://huggingface.co/docs/trl/sft_trainer#using-tokenids-directly-for-responsetemplate

In [13]:
response_template = "### Response:"
# collator = DataCollatorForCompletionOnlyLM(response_template, tokenizer=tokenizer)
collator = DataCollatorForCompletionOnlyLM(tokenizer.encode(response_template, add_special_tokens = False)[2:], tokenizer=tokenizer)


In [14]:
trainer = SFTTrainer(
                model = model,
                train_dataset = train_dataset,
                eval_dataset = val_dataset,
                data_collator = collator,
                peft_config = peft_config,                
                dataset_text_field = "text",
                max_seq_length = 512,
                tokenizer = tokenizer,
                args = training_params,
                packing = False, # To not pack examples to fill seq len
            )

Map: 100%|██████████| 852/852 [00:00<00:00, 7054.65 examples/s]
Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
/bin/sh: 1: scontrol: not found
[codecarbon WARNING @ 20:00:35] Error running `scontrol show job $SLURM_JOBID` to count SLURM-available cpus. Using the machine's cpu count.
[codecarbon INFO @ 20:00:35] [setup] RAM Tracking...
/bin/sh: 1: scontrol: not found
[codecarbon WARNING @ 20:00:35] Error running `scontrol show job $SLURM_JOBID` to retrieve SLURM-available RAM.Using the machine's total RAM.
[codecarbon INFO @ 20:00:35] [setup] GPU Tracking...
[codecarbon INFO @ 20:00:35] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 20:00:35] [setup] CPU Tracking...
[codecarbon INFO @ 20:00:36] Tracking Intel CPU via RAPL interface
[codecarbon INFO @ 20:00:38] >>> Tracker's metadata:
[codecarbon INFO @ 20:00:38]   Platform system: Linux-4.18.

In [15]:
train_dataset[0]

{'Unnamed: 0': 6587,
 'id': 169269,
 'tweet_id': 1.58569e+18,
 'aggression': 0,
 'offense': 0,
 'codemixed': 1,
 'tweet_text': "Let's get some zimbabwe players into ipl and invite them to play series against india in india #zimbabwe #PAKvsZIM #T20worldcup22",
 'text': "You are an expert in hate speech detection. Offensive tweets are defined as tweets containing profane words, sarcastic remarks, insults, slanders or slurs. These can have a potentially harmful effect on a given target. Classify the following input tweet as Offensive or Non-Offensive.\n\n ### Input: Let's get some zimbabwe players into ipl and invite them to play series against india in india #zimbabwe #PAKvsZIM #T20worldcup22\n\n ### Response: Non-Offensive"}

In [ ]:
trainer.train()

You are using 8-bit optimizers with a version of `bitsandbytes` < 0.41.1. It is recommended to update your version as a major bug has been fixed in 8-bit optimizers.
You're using a BloomTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
1,0.124100,0.196209
2,0.138000,0.161415
3,0.152500,0.214209


[codecarbon WARNING @ 20:00:57] Error running `scontrol show job $SLURM_JOBID` to retrieve SLURM-available RAM.Using the machine's total RAM.
/bin/sh: 1: scontrol: not found
[codecarbon INFO @ 20:00:57] Energy consumed for RAM : 0.000590 kWh. RAM Power : 141.4961543083191 W
[codecarbon WARNING @ 20:00:57] Error running `scontrol show job $SLURM_JOBID` to retrieve SLURM-available RAM.Using the machine's total RAM.
/bin/sh: 1: scontrol: not found
[codecarbon INFO @ 20:00:57] Energy consumed for all GPUs : 0.001079 kWh. Total GPU Power : 258.733 W
[codecarbon INFO @ 20:00:57] Energy consumed for all CPUs : 0.001412 kWh. Total CPU Power : 338.40557480045334 W
[codecarbon INFO @ 20:00:57] 0.003081 kWh of electricity used since the beginning.
/bin/sh: 1: scontrol: not found
[codecarbon WARNING @ 20:01:12] Error running `scontrol show job $SLURM_JOBID` to retrieve SLURM-available RAM.Using the machine's total RAM.
[codecarbon INFO @ 20:01:12] Energy consumed for RAM : 0.001178 kWh. RAM Power 

/bin/sh: 1: scontrol: not found
[codecarbon WARNING @ 20:03:42] Error running `scontrol show job $SLURM_JOBID` to retrieve SLURM-available RAM.Using the machine's total RAM.
[codecarbon INFO @ 20:03:42] Energy consumed for RAM : 0.007064 kWh. RAM Power : 141.4961543083191 W
[codecarbon WARNING @ 20:03:42] Error running `scontrol show job $SLURM_JOBID` to retrieve SLURM-available RAM.Using the machine's total RAM.
/bin/sh: 1: scontrol: not found
[codecarbon INFO @ 20:03:42] Energy consumed for all GPUs : 0.012484 kWh. Total GPU Power : 247.96200000000002 W
[codecarbon INFO @ 20:03:42] Energy consumed for all CPUs : 0.017374 kWh. Total CPU Power : 364.5405371505589 W
[codecarbon INFO @ 20:03:42] 0.036921 kWh of electricity used since the beginning.
[codecarbon WARNING @ 20:03:57] Error running `scontrol show job $SLURM_JOBID` to retrieve SLURM-available RAM.Using the machine's total RAM.
/bin/sh: 1: scontrol: not found
[codecarbon INFO @ 20:03:57] Energy consumed for RAM : 0.007653 kWh. 

/bin/sh: 1: scontrol: not found
[codecarbon WARNING @ 20:06:27] Error running `scontrol show job $SLURM_JOBID` to retrieve SLURM-available RAM.Using the machine's total RAM.
[codecarbon INFO @ 20:06:27] Energy consumed for RAM : 0.013540 kWh. RAM Power : 141.4961543083191 W
/bin/sh: 1: scontrol: not found
[codecarbon WARNING @ 20:06:27] Error running `scontrol show job $SLURM_JOBID` to retrieve SLURM-available RAM.Using the machine's total RAM.
[codecarbon INFO @ 20:06:27] Energy consumed for all GPUs : 0.023904 kWh. Total GPU Power : 250.48700000000002 W
[codecarbon INFO @ 20:06:27] Energy consumed for all CPUs : 0.033275 kWh. Total CPU Power : 345.65292791005504 W
[codecarbon INFO @ 20:06:27] 0.070718 kWh of electricity used since the beginning.
/bin/sh: 1: scontrol: not found
[codecarbon WARNING @ 20:06:42] Error running `scontrol show job $SLURM_JOBID` to retrieve SLURM-available RAM.Using the machine's total RAM.
[codecarbon INFO @ 20:06:42] Energy consumed for RAM : 0.014128 kWh.

...

[codecarbon WARNING @ 22:22:57] Error running `scontrol show job $SLURM_JOBID` to retrieve SLURM-available RAM.Using the machine's total RAM.
/bin/sh: 1: scontrol: not found
[codecarbon INFO @ 22:22:57] Energy consumed for RAM : 0.334949 kWh. RAM Power : 141.4961543083191 W
[codecarbon WARNING @ 22:22:57] Error running `scontrol show job $SLURM_JOBID` to retrieve SLURM-available RAM.Using the machine's total RAM.
/bin/sh: 1: scontrol: not found
[codecarbon INFO @ 22:22:57] Energy consumed for all GPUs : 0.586573 kWh. Total GPU Power : 247.338 W
[codecarbon INFO @ 22:22:57] Energy consumed for all CPUs : 0.821123 kWh. Total CPU Power : 360.8216940028898 W
[codecarbon INFO @ 22:22:57] 1.742645 kWh of electricity used since the beginning.
[codecarbon WARNING @ 22:23:12] Error running `scontrol show job $SLURM_JOBID` to retrieve SLURM-available RAM.Using the machine's total RAM.
/bin/sh: 1: scontrol: not found
[codecarbon INFO @ 22:23:12] Energy consumed for RAM : 0.335538 kWh. RAM Power :

/bin/sh: 1: scontrol: not found
[codecarbon WARNING @ 22:25:42] Error running `scontrol show job $SLURM_JOBID` to retrieve SLURM-available RAM.Using the machine's total RAM.
[codecarbon INFO @ 22:25:42] Energy consumed for RAM : 0.341425 kWh. RAM Power : 141.4961543083191 W
[codecarbon WARNING @ 22:25:42] Error running `scontrol show job $SLURM_JOBID` to retrieve SLURM-available RAM.Using the machine's total RAM.
/bin/sh: 1: scontrol: not found
[codecarbon INFO @ 22:25:42] Energy consumed for all GPUs : 0.597852 kWh. Total GPU Power : 256.166 W
[codecarbon INFO @ 22:25:42] Energy consumed for all CPUs : 0.836630 kWh. Total CPU Power : 343.18737367072094 W
[codecarbon INFO @ 22:25:42] 1.775907 kWh of electricity used since the beginning.
[codecarbon WARNING @ 22:25:57] Error running `scontrol show job $SLURM_JOBID` to retrieve SLURM-available RAM.Using the machine's total RAM.
/bin/sh: 1: scontrol: not found
[codecarbon INFO @ 22:25:57] Energy consumed for RAM : 0.342014 kWh. RAM Power 

[codecarbon WARNING @ 22:29:27] Error running `scontrol show job $SLURM_JOBID` to retrieve SLURM-available RAM.Using the machine's total RAM.
[codecarbon INFO @ 22:29:27] Energy consumed for RAM : 0.350255 kWh. RAM Power : 141.4961543083191 W
[codecarbon WARNING @ 22:29:27] Error running `scontrol show job $SLURM_JOBID` to retrieve SLURM-available RAM.Using the machine's total RAM.
/bin/sh: 1: scontrol: not found
/bin/sh: 1: scontrol: not found
[codecarbon INFO @ 22:29:27] Energy consumed for all GPUs : 0.613414 kWh. Total GPU Power : 258.949 W
[codecarbon INFO @ 22:29:27] Energy consumed for all CPUs : 0.857915 kWh. Total CPU Power : 348.50349121905253 W
[codecarbon INFO @ 22:29:27] 1.821583 kWh of electricity used since the beginning.
[codecarbon WARNING @ 22:29:42] Error running `scontrol show job $SLURM_JOBID` to retrieve SLURM-available RAM.Using the machine's total RAM.
/bin/sh: 1: scontrol: not found
[codecarbon INFO @ 22:29:42] Energy consumed for RAM : 0.350843 kWh. RAM Power 

[codecarbon WARNING @ 22:33:42] Error running `scontrol show job $SLURM_JOBID` to retrieve SLURM-available RAM.Using the machine's total RAM.
/bin/sh: 1: scontrol: not found
[codecarbon INFO @ 22:33:42] Energy consumed for RAM : 0.360262 kWh. RAM Power : 141.4961543083191 W
[codecarbon WARNING @ 22:33:42] Error running `scontrol show job $SLURM_JOBID` to retrieve SLURM-available RAM.Using the machine's total RAM.
/bin/sh: 1: scontrol: not found
[codecarbon INFO @ 22:33:42] Energy consumed for all GPUs : 0.630988 kWh. Total GPU Power : 195.33700000000002 W
[codecarbon INFO @ 22:33:42] Energy consumed for all CPUs : 0.881999 kWh. Total CPU Power : 335.76573273717725 W
[codecarbon INFO @ 22:33:42] 1.873250 kWh of electricity used since the beginning.
/bin/sh: 1: scontrol: not found
[codecarbon WARNING @ 22:33:57] Error running `scontrol show job $SLURM_JOBID` to retrieve SLURM-available RAM.Using the machine's total RAM.
[codecarbon INFO @ 22:33:57] Energy consumed for RAM : 0.360851 kWh.

[codecarbon INFO @ 22:36:12] 1.903428 kWh of electricity used since the beginning.
[codecarbon WARNING @ 22:36:27] Error running `scontrol show job $SLURM_JOBID` to retrieve SLURM-available RAM.Using the machine's total RAM.
/bin/sh: 1: scontrol: not found
[codecarbon INFO @ 22:36:27] Energy consumed for RAM : 0.366736 kWh. RAM Power : 141.4961543083191 W
/bin/sh: 1: scontrol: not found
[codecarbon WARNING @ 22:36:27] Error running `scontrol show job $SLURM_JOBID` to retrieve SLURM-available RAM.Using the machine's total RAM.
[codecarbon INFO @ 22:36:27] Energy consumed for all GPUs : 0.642135 kWh. Total GPU Power : 254.217 W
[codecarbon INFO @ 22:36:27] Energy consumed for all CPUs : 0.897593 kWh. Total CPU Power : 333.9693479302807 W
[codecarbon INFO @ 22:36:27] 1.906464 kWh of electricity used since the beginning.
/bin/sh: 1: scontrol: not found
[codecarbon WARNING @ 22:36:42] Error running `scontrol show job $SLURM_JOBID` to retrieve SLURM-available RAM.Using the machine's total RA

In [ ]:
trainer.state.log_history

## Inference

In [4]:
device = torch.device("cuda") if torch.cuda.is_available() else "cpu"

In [5]:
trained_checkpoint = "results_bloomz/checkpoint-5109/"

tokenizer = AutoTokenizer.from_pretrained(trained_checkpoint)
model = AutoModelForCausalLM.from_pretrained(trained_checkpoint)
model = model.to(device)
model.eval()

BloomForCausalLM(
  (transformer): BloomModel(
    (word_embeddings): Embedding(250880, 4096)
    (word_embeddings_layernorm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
    (h): ModuleList(
      (0-29): 30 x BloomBlock(
        (input_layernorm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (self_attention): BloomAttention(
          (query_key_value): Linear(
            in_features=4096, out_features=12288, bias=True
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.1, inplace=False)
            )
            (lora_A): ModuleDict(
              (default): Linear(in_features=4096, out_features=64, bias=False)
            )
            (lora_B): ModuleDict(
              (default): Linear(in_features=64, out_features=12288, bias=False)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
          )
          (dense): Linear(in_features=4096, out_features=4096, bias=True)

In [6]:
val_df = pd.read_csv("data/splits/val.csv")
test_df = pd.read_csv("data/splits/test.csv")

In [7]:
val_df["text"] = val_df.apply(lambda row: prepare_prompt(row, train=False), axis=1)
test_df["text"] = test_df.apply(lambda row: prepare_prompt(row, train=False), axis=1)

val_df["text"].values[:2]

array(['You are an expert in hate speech detection. Offensive tweets are defined as tweets containing profane words, sarcastic remarks, insults, slanders or slurs. These can have a potentially harmful effect on a given target. Classify the following input tweet as Offensive or Non-Offensive.\n\n ### Input: @user Congress is not a political party.. It is a INC Pvt. Ltd. made by royal Gandhi family for loot people and build new scams.. @user @user @user \n\n@user @user\n\n ### Response: ',
       'You are an expert in hate speech detection. Offensive tweets are defined as tweets containing profane words, sarcastic remarks, insults, slanders or slurs. These can have a potentially harmful effect on a given target. Classify the following input tweet as Offensive or Non-Offensive.\n\n ### Input: @user लगता है सरकार कोई है ही नही...इसपे UAPA लगना चाहिए और साथ ही इसके घर पे बुलडोझर चलना चाहिए..\n\n ### Response: '],
      dtype=object)

In [8]:
def inference_responses(df):
    model.eval()
    responses = []

    for i in tqdm(range(len(df))):
        inputs = tokenizer(df["text"][i], padding=True, truncation=True, max_length=256, 
                           return_tensors="pt").to(device)
        with torch.no_grad():
            generate_ids = model.generate(inputs.input_ids, max_length=256)

        response = tokenizer.batch_decode(generate_ids, skip_special_tokens=True, 
                                          clean_up_tokenization_spaces=False)[0]
        responses.append(response)
    return responses

In [9]:
def get_labels(responses):
    labels = []
    response_trimmed = 0
    label_absent = 0

    for response in responses:
        splitted = response.split("### Response: ")
        if len(splitted) == 1:
            #print(response, "\n")
            response_trimmed += 1
            label = 0 #-1
            
        else:
            if "Non-Offensive" in splitted[1][:15]:
                label = 0
            elif "Offensive" in splitted[1][:15]:
                label = 1
            else:
                label_absent += 1
                label = 0 # Default majority class
                
        labels.append(label)

    print(f"{response_trimmed} responses trimmed due to max_length")
    print(f"{label_absent} labels absent \n")
    return labels

In [10]:
def print_metrics(labels, df):
    print("F1 score = ", f1_score(df['offense'].tolist(), labels))
    print(classification_report(df['offense'].tolist(), labels, target_names=["Non-Offensive (0)", "Offensive (1)"]))

In [11]:
val_responses = inference_responses(val_df)
val_labels = get_labels(val_responses)
print_metrics(val_labels, val_df)

100%|██████████| 852/852 [04:04<00:00,  3.49it/s]

0 responses trimmed due to max_length
0 labels absent 

F1 score =  0.6885964912280701
                   precision    recall  f1-score   support

Non-Offensive (0)       0.85      0.93      0.89       596
    Offensive (1)       0.79      0.61      0.69       256

         accuracy                           0.83       852
        macro avg       0.82      0.77      0.79       852
     weighted avg       0.83      0.83      0.83       852



In [12]:
with open('data/predictions/bloomz-ft-completion_val.pickle', 'wb') as f:
    pickle.dump(val_labels, f, protocol=pickle.HIGHEST_PROTOCOL)

# with open('data/predictions/llama-ft-completion_val.pickle', 'rb') as handle:
#     val_labels = pickle.load(handle)

In [13]:
test_responses = inference_responses(test_df)
test_labels = get_labels(test_responses)
print_metrics(test_labels, test_df)

with open('data/predictions/bloomz-ft-completion_test.pickle', 'wb') as f:
    pickle.dump(test_labels, f, protocol=pickle.HIGHEST_PROTOCOL)

 88%|████████▊ | 752/851 [03:27<00:31,  3.19it/s]/home/sn3250/.local/lib/python3.10/site-packages/transformers/generation/utils.py:1268: UserWarning: Input length of input_ids is 256, but `max_length` is set to 256. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(
100%|██████████| 851/851 [03:54<00:00,  3.63it/s]

1 responses trimmed due to max_length
0 labels absent 

F1 score =  0.6387665198237886
                   precision    recall  f1-score   support

Non-Offensive (0)       0.83      0.91      0.87       595
    Offensive (1)       0.73      0.57      0.64       256

         accuracy                           0.81       851
        macro avg       0.78      0.74      0.75       851
     weighted avg       0.80      0.81      0.80       851

